#Identification of Employee Turnover Risk using Machine Learning: A Path to Enhanced Organizational Insights for Retention Strategies

Technical Notebook

In [44]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import pandas as pd
import numpy as np
import sqlite3
import csv

## Create the SQLITE3 Database, Tables and INSERT records from .csv files

In [51]:
#connect to a sqlite3 database, or create one if it does not exist
conn = sqlite3.connect('/content/drive/My Drive/MS-ADS Capstone Project/data/hr_case_study.db')

#create the cursor object
cursor = conn.cursor()

#execute CREATE TABLE commands to create the required tables in the hr_case_study.db

#create employee_survey_data table
cursor.execute('''CREATE TABLE IF NOT EXISTS employee_survey_data
                (EmployeeID TEXT PRIMARY KEY, EnvironmentSatisfaction TEXT, JobSatisfaction TEXT, WorkLifeBalance TEXT)''')

#create manager_survey_data table
cursor.execute('''CREATE TABLE IF NOT EXISTS manager_survey_data
                (EmployeeID TEXT PRIMARY KEY, JobInvolvement TEXT, PerformanceRating TEXT)''')

#create general_data table
cursor.execute('''CREATE TABLE IF NOT EXISTS general_data
                (Age INT, Attrition TEXT, BusinessTravel TEXT, Department TEXT, DistanceFromHome INT, Education INT, EducationField TEXT, EmployeeCount INT, EmployeeID TEXT,
                Gender TEXT, JobLevel INT, JobRole TEXT, MaritalStatus TEXT, MonthlyIncome INT, NumCompaniesWorked TEXT, Over18 TEXT, PercentSalaryHike INT, 
                StandardHours INT, StockOptionLevel INT, TotalWorkingYears TEXT, TrainingTimesLastYear TEXT, YearsAtCompany TEXT, YearsSinceLastPromotion TEXT, YearsWithCurrentManager TEXT)''')

#save the changes to the database
conn.commit()

##### Write the data from .csv files to each table ##### 

with open('/content/drive/My Drive/MS-ADS Capstone Project/data/employee_survey_data.csv', 'r') as csvfile:
    #read contents of .csv file and insert data into SQLite database
    csvreader = csv.reader(csvfile)
    #skip the first row of data
    next(csvreader)
    for row in csvreader:
        cursor.execute("INSERT INTO employee_survey_data (EmployeeID, EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance) VALUES (?, ?, ?, ?)", (row[0], row[1], row[2], row[3]))
#save changes and commit to database
conn.commit()

with open('/content/drive/My Drive/MS-ADS Capstone Project/data/manager_survey_data.csv', 'r') as csvfile:
    #read contents of .csv file and insert data into SQLite database
    csvreader = csv.reader(csvfile)
    #skip the first row of data
    next(csvreader)
    for row in csvreader:
        cursor.execute("INSERT INTO manager_survey_data (EmployeeID, JobInvolvement, PerformanceRating) VALUES (?, ?, ?)", (row[0], row[1], row[2]))
#save changes and commit to database
conn.commit()

with open('/content/drive/My Drive/MS-ADS Capstone Project/data/general_data.csv', 'r') as csvfile:
    #read contents of .csv file and insert data into SQLite database
    csvreader = csv.reader(csvfile)
    #skip the first row of data
    next(csvreader)
    for row in csvreader:
        cursor.execute("""INSERT INTO general_data (Age, Attrition, BusinessTravel, Department, DistanceFromHome, Education, EducationField, EmployeeCount, 
        EmployeeID, Gender, JobLevel, JobRole, MaritalStatus, MonthlyIncome, NumCompaniesWorked, Over18, PercentSalaryHike, StandardHours, StockOptionLevel, 
        TotalWorkingYears, TrainingTimesLastYear, YearsAtCompany, YearsSinceLastPromotion, YearsWithCurrentManager) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
                       (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], 
                        row[16], row[17], row[18],row[19], row[20], row[21], row[22], row[23]))
#save changes and commit to database
conn.commit()

#close the cursor and connection objects
cursor.close()
conn.close()

### Query the Database

#### SELECT all records from general_data TABLE

In [52]:
#connect to the database
conn = sqlite3.connect('/content/drive/My Drive/MS-ADS Capstone Project/data/hr_case_study.db')

##### Query general_data table ##### 
#create cursor object and query table
cursor = conn.cursor()
cursor.execute('SELECT * FROM general_data')
col_names = [desc[0] for desc in cursor.description]

#fetch all rows from the query and store in dataframe
rows = cursor.fetchall()
general_df = pd.DataFrame(rows, columns = col_names)

#close the cursor and connection objects
cursor.close()
conn.close()

#### SELECT all records from employee_survey_data TABLE

In [55]:
#connect to the database
conn = sqlite3.connect('/content/drive/My Drive/MS-ADS Capstone Project/data/hr_case_study.db')

##### Query general_data table ##### 
#create cursor object and query table
cursor = conn.cursor()
cursor.execute('SELECT * FROM employee_survey_data')
col_names = [desc[0] for desc in cursor.description]

#fetch all rows from the query and store in dataframe
rows = cursor.fetchall()
employee_survey_df = pd.DataFrame(rows, columns = col_names)

#close the cursor and connection objects
cursor.close()
conn.close()

#### SELECT all records from the manger_survey_data TABLE

In [56]:
#connect to the database
conn = sqlite3.connect('/content/drive/My Drive/MS-ADS Capstone Project/data/hr_case_study.db')

##### Query general_data table ##### 
#create cursor object and query table
cursor = conn.cursor()
cursor.execute('SELECT * FROM manager_survey_data')
col_names = [desc[0] for desc in cursor.description]

#fetch all rows from the query and store in dataframe
rows = cursor.fetchall()
manager_survey_df = pd.DataFrame(rows, columns = col_names)

#close the cursor and connection objects
cursor.close()
conn.close()

#### Check each dataframe for correct importing

In [57]:
general_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrentManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1,Y,11,8,0,1,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0,Y,23,8,1,6,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1,Y,15,8,3,5,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3,Y,11,8,3,13,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4,Y,12,8,2,9,2,6,0,4


In [58]:
employee_survey_df.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3,4,2
1,2,3,2,4
2,3,2,2,1
3,4,4,4,3
4,5,4,1,3


In [59]:
manager_survey_df.head()

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [53]:
print(general_df.shape)
general_df.head()

(4410, 24)


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrentManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1,Y,11,8,0,1,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0,Y,23,8,1,6,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1,Y,15,8,3,5,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3,Y,11,8,3,13,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4,Y,12,8,2,9,2,6,0,4


In [54]:
general_df.dtypes

Age                         int64
Attrition                  object
BusinessTravel             object
Department                 object
DistanceFromHome            int64
Education                   int64
EducationField             object
EmployeeCount               int64
EmployeeID                 object
Gender                     object
JobLevel                    int64
JobRole                    object
MaritalStatus              object
MonthlyIncome               int64
NumCompaniesWorked         object
Over18                     object
PercentSalaryHike           int64
StandardHours               int64
StockOptionLevel            int64
TotalWorkingYears          object
TrainingTimesLastYear      object
YearsAtCompany             object
YearsSinceLastPromotion    object
YearsWithCurrentManager    object
dtype: object